# CSDA with Lindberg Operators

Annotated Version in Jupyter from program CSDALindberg.jl. 

This notebook requires a HDF5 file that cointains the 
LFP, and a list of unusable electrodes.

In [3]:
push!(LOAD_PATH, ".")

5-element Array{String,1}:
 "@"
 "@v#.#"
 "@stdlib"
 "."
 "."

In [5]:
using HDF5
using LindbergOperators, ParaSets
#importall LindenbergOperadores

┌ Info: Precompiling LindbergOperators [top-level]
└ @ Base loading.jl:1260
┌ Info: Precompiling ParaSets [top-level]
└ @ Base loading.jl:1260


In [6]:
arxnombre="../BRWFiles/test.h5"
arx=h5open(arxnombre)
names(arx)

HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 0:
  #000: H5F.c line 370 in H5Fis_hdf5(): unable open file
    major: File accessibilty
    minor: Not an HDF5 file
  #001: H5Fint.c line 802 in H5F__is_hdf5(): unable to open file
    major: Low-level I/O
    minor: Unable to initialize object
  #002: H5FD.c line 734 in H5FD_open(): open failed
    major: Virtual File Layer
    minor: Unable to initialize object
  #003: H5FDsec2.c line 346 in H5FD_sec2_open(): unable to open file: name = '../BRWFiles/test.h5', errno = 2, error message = 'No existe el fichero o el directorio', flags = 0, o_flags = 0
    major: File accessibilty
    minor: Unable to open file


ErrorException: Cannot access file ../BRWFiles/test.h5

In [9]:
# Check how you named the LFP data and put the right name here
LFPname="LFPSaturados"
    LFP=read(arx["LFPname"])
# or maybe
# LFP=read(arx["LFPTotal"])

saturadosarray=read(arx["CanalesSaturados"])
respuestasarray=read(arx["Canalesrespuesta"]);
frecuencia=read(arx["freq"])
close(arx)

UndefVarError: UndefVarError: arx not defined

In [24]:
saturados=arraytoset(saturadosarray)
respuestas=arraytoset(respuestasarray)

Set(Any[[25, 55], [22, 48], [19, 59], [48, 35], [20, 52], [17, 50], [25, 51], [23, 58], [21, 54], [20, 48]  …  [20, 54], [31, 53], [22, 52], [27, 52], [18, 57], [25, 52], [19, 57], [25, 59], [23, 51], [20, 50], [20, 56]])

In [25]:


lfpParchado=copy(LFP)
(l,m,cmax)=size(lfpParchado)
# Set to zero unusable channels
for m in saturados
    q=m[1]
    p=m[2]
    lfpParchado[q,p,:]=zeros(cmax)
end

In [26]:
listaredux=TiraOrillas(saturados)


Set(Any[[29, 58], [34, 7], [29, 55], [33, 60], [38, 61], [32, 57], [27, 61], [25, 51], [33, 61], [36, 63]  …  [46, 57], [37, 60], [29, 40], [29, 5], [35, 49], [36, 57], [31, 61], [34, 56], [23, 52], [33, 59], [36, 59]])

In [27]:
#Let us put a smoothing over the new zeros.
for m in listaredux
        q=m[1]
        p=m[2]
        vecinos=vecindad8(m)
        lfpParchado[q,p,:]=promediasobreconjunto(vecinos,lfpParchado)
end


In [28]:
size(lfpParchado)

(64, 64, 459)

In [29]:
(mu,nu,lu)=size(lfpParchado)
#Gaussian Temporal Smoothing, makes animations clear.
lfpplanchado=zeros(mu,nu,lu)
for j=1:mu,l=1:nu
    porromponpon=vec(lfpParchado[j,l,:])
    lfpplanchado[j,l,:]=GaussSuavizarTemporal(porromponpon)
end



In [14]:

aux1=zeros(mu,nu,lu)
aux2=zeros(mu,nu,lu)
#Spatial Gaussian Smoothing and dCSD.

for t=1:lu
    aux1[:,:,t]=GaussianSmooth(lfpplanchado[:,:,t])
    aux2[:,:,t]=DiscreteLaplacian(aux1[:,:,t])
end
CSD=-aux2;


In [30]:
# saving file
h5open(arxnombre, "r+") do file
    write(file, "CSDALindenberg", CSD)  # alternatively, say "@write file A"
end


In [10]:
#check file
arx=h5open(arxnombre)
names(arx)

HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 0:
  #000: H5F.c line 370 in H5Fis_hdf5(): unable open file
    major: File accessibilty
    minor: Not an HDF5 file
  #001: H5Fint.c line 802 in H5F__is_hdf5(): unable to open file
    major: Low-level I/O
    minor: Unable to initialize object
  #002: H5FD.c line 734 in H5FD_open(): open failed
    major: Virtual File Layer
    minor: Unable to initialize object
  #003: H5FDsec2.c line 346 in H5FD_sec2_open(): unable to open file: name = '../BRWFiles/test.h5', errno = 2, error message = 'No existe el fichero o el directorio', flags = 0, o_flags = 0
    major: File accessibilty
    minor: Unable to open file


ErrorException: Cannot access file ../BRWFiles/test.h5

In [32]:
close(arx)